In [7]:
import pandas as pd
import numpy as np
from datetime import datetime

df = pd.read_csv('../data/external/vacinacao-se.csv', sep=';', parse_dates=['paciente_datanascimento', 'vacina_dataaplicacao'])
age_groups = [(0, 17), (18, 35), (36, 59), (60, 200)]

MemoryError: Unable to allocate 128. KiB for an array with shape (16384,) and data type int64

In [ ]:
df.columns

Index(['document_id', 'paciente_id', 'paciente_idade',
       'paciente_datanascimento', 'paciente_enumsexobiologico',
       'paciente_racacor_codigo', 'paciente_racacor_valor',
       'paciente_endereco_coibgemunicipio', 'paciente_endereco_copais',
       'paciente_endereco_nmmunicipio', 'paciente_endereco_nmpais',
       'paciente_endereco_uf', 'paciente_endereco_cep',
       'paciente_nacionalidade_enumnacionalidade', 'estabelecimento_valor',
       'estabelecimento_razaosocial', 'estalecimento_nofantasia',
       'estabelecimento_municipio_codigo', 'estabelecimento_municipio_nome',
       'estabelecimento_uf', 'vacina_grupoatendimento_codigo',
       'vacina_grupoatendimento_nome', 'vacina_categoria_codigo',
       'vacina_categoria_nome', 'vacina_lote', 'vacina_fabricante_nome',
       'vacina_fabricante_referencia', 'vacina_dataaplicacao',
       'vacina_descricao_dose', 'vacina_codigo', 'vacina_nome',
       'sistema_origem', 'data_importacao_rnds', 'id_sistema_origem'],
      

In [ ]:
df = df.assign(vacinacao_completa=(df.vacina_descricao_dose == '2ª Dose').where((df.vacina_fabricante_nome != 'JANSSEN'), True))

In [ ]:
dfs = []
for i in range(1, 13):
    date = datetime.strptime('01-' + str(i) + '-2021', '%d-%m-%Y')
    
    df_ages = None

    for group in age_groups:
        temp1 = df[(df['paciente_idade'] > group[0]) & (df['paciente_idade'] < group[1]) &
                  (df['vacina_dataaplicacao'] < date) & (df['vacinacao_completa'] == False)].groupby(['estabelecimento_municipio_nome']).size().to_frame('vacinados_parcial_' + str(group[1]))
                  

        temp2 = df[(df['paciente_idade'] > group[0]) & (df['paciente_idade'] < group[1]) &
                  (df['vacina_dataaplicacao'] < date) & (df['vacinacao_completa'] == True)].groupby(['estabelecimento_municipio_nome']).size().to_frame('vacinados_completo_' + str(group[1]))
        
        tempF = df[(df['paciente_enumsexobiologico']=='F')].groupby(['estabelecimento_municipio_nome']).size().to_frame('vacina_F') & (df['vacina_dataaplicacao'] < date)

        
        temp3 = temp1.merge(temp2, on='estabelecimento_municipio_nome')
        temp = tempF.merge(temp3, on= 'estabelecimento_municipio_nome')

        if df_ages is not None:
            df_ages = df_ages.merge(temp, on='estabelecimento_municipio_nome')
        else:
            df_ages = temp

    df_ages['date'] = date
    dfs.append(df_ages)
final_df = pd.concat(dfs)

MemoryError: Unable to allocate 179. MiB for an array with shape (187563825,) and data type bool

In [ ]:
final_df

,vacinados_parcial_17,vacinados_completo_17,vacinados_parcial_35,vacinados_completo_35,vacinados_parcial_59,vacinados_completo_59,vacinados_parcial_200,vacinados_completo_200,date
estabelecimento_municipio_nome,,,,,,,,,
ARACAJU,4,1,5182,3037,8339,4353,3067,426,2021-03-01
NOSSA SENHORA DA GLORIA,1,1,139,67,264,102,143,22,2021-03-01
ARACAJU,10,1,7439,4598,10925,6444,41837,2423,2021-04-01
NOSSA SENHORA DA GLORIA,1,1,258,84,398,166,2346,610,2021-04-01
ARACAJU,13,6,11314,6374,15964,8399,75883,19176,2021-05-01
...,...,...,...,...,...,...,...,...,...
SANTA LUZIA DO ITANHY,800,1,3287,2127,3384,2934,1461,1279,2021-12-01
SANTO AMARO DAS BROTAS,2,1,151,61,840,129,1023,265,2021-12-01
SAO CRISTOVAO,3,1,5512,285,4313,1328,1562,173,2021-12-01


In [ ]:
final_df.to_csv('final_vacinacao.csv', index=False)